In [1]:
#import dependencies

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veeranandhanvj/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
#Printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
#Data preprocessing

#Loading the news dataset

news_data = pd.read_csv('NewsData.csv')
print(news_data.shape)

(20800, 5)


In [21]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [22]:
#Counting the number of null values

news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [23]:
#Replacing the null values with empty string

news_data = news_data.fillna('')

In [24]:
news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [29]:
#Merging the title and author

news_data['content'] = news_data['author']+' '+news_data['title']
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [31]:
#separate data and lable

X= news_data.drop(columns='label', axis=1)
Y= news_data['label']
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [32]:
#Stemming

port_stem = PorterStemmer()

In [36]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [37]:
news_data['content']= news_data['content'].apply(stemming)

In [41]:
news_data['content']

0        darrel lucu hous dem aid we didn even see come...
1        daniel j flynn flynn hillari clinton big woman...
2           consortiumnew com whi truth might get you fire
3        jessica purkiss civilian kill in singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t i trump poster child for...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci is said...
20798    alex ansari nato russia to hold parallel exerc...
20799                       david swanson what keep f aliv
Name: content, Length: 20800, dtype: object

In [42]:
#separate data and lable

X= news_data['content'].values
Y= news_data['label'].values
print(X)
print(Y)

['darrel lucu hous dem aid we didn even see comey letter until jason chaffetz tweet it'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get you fire' ...
 'michael j de la merc rachel abram maci is said receiv takeov approach hudson bay the new york time'
 'alex ansari nato russia to hold parallel exercis in balkan'
 'david swanson what keep f aliv']
[1 0 1 ... 0 1 1]


In [43]:
#Converting text data to numeric data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X= vectorizer.transform(X)

In [48]:
print(X)

  (0, 272)	0.23876842918538388
  (0, 2501)	0.3250028762214838
  (0, 2977)	0.21821000835755874
  (0, 3619)	0.31814479104571186
  (0, 3811)	0.23915031396884365
  (0, 4028)	0.26679080472409034
  (0, 4998)	0.20625126961544932
  (0, 7040)	0.19336678366028096
  (0, 7639)	0.16112384433475346
  (0, 7731)	0.21910035354703797
  (0, 8670)	0.2582374612067049
  (0, 8949)	0.3214177525383041
  (0, 13528)	0.22682424464275372
  (0, 15762)	0.2518068275408432
  (0, 16055)	0.2905362997207957
  (0, 16586)	0.19235766814206554
  (1, 1512)	0.2939891562094648
  (1, 1910)	0.15521974226349364
  (1, 2241)	0.3827320386859759
  (1, 2831)	0.19094574062359204
  (1, 3587)	0.26373768806048464
  (1, 5528)	0.7143299355715573
  (1, 6849)	0.1904660198296849
  (1, 16892)	0.30071745655510157
  (2, 2961)	0.2871371098339331
  :	:
  (20797, 9561)	0.29181282737961417
  (20797, 9631)	0.1724218928119192
  (20797, 10352)	0.07625035854083997
  (20797, 12192)	0.24475674109592507
  (20797, 12398)	0.2693052563625126
  (20797, 13176)	0.

In [45]:
#Splittig dataset to train test data

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [46]:
print(x_train.shape)

(16640, 17227)


In [50]:
#Training the model

model = LogisticRegression()

In [51]:
model.fit(x_train,y_train)

LogisticRegression()

In [53]:
#Model evaluation

#Accuracy score on training data

x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)
print("Accuracy score on training data : ", training_data_accuracy)

Accuracy score on training data :  0.98671875


In [54]:
#Accuracy score on test data

x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)
print("Accuracy score on test data : ", test_data_accuracy)

Accuracy score on test data :  0.9764423076923077


In [58]:
#Making a predictive system

x_row_data = x_test[20]

prediction = model.predict(x_row_data)

print(prediction)

if prediction:
    print("The news is fake")
else:
    print("The news is real")

[0]
The news is real


In [59]:
print(y_test[20])

0
